***
### Import of required libraries
***

In [1]:
import glob
from tqdm.auto import tqdm
import pandas as pd

***
### Import of data
***

##### Import of M1 data

In [2]:
# Get a list of all the m1 so6 files
filelist = glob.glob("/store/Projects_CRM/RAD_paper/RAD_new_data/m1/*.so6")

# Create a list to store the daily dataframes
read_files = []

# Loop through the files and read them into a dataframe which is then appended
# to the list
for filename in tqdm(filelist):
    day = pd.read_csv(
        filename,
        sep=" ",
        header=None,
        index_col=False,
        names=[
            "segment_id",
            "origin",
            "destination",
            "ac_type",
            "segment_start_t",
            "segment_stop_t",
            "segment_start_fl",
            "segment_stop_fl",
            "segment_status",
            "callsign",
            "segment_start_d",
            "segment_stop_d",
            "segment_start_lat",
            "segment_start_lon",
            "segment_stop_lat",
            "segment_stop_lon",
            "identifier",
            "sequence",
            "segment_length_nm",
            "segment_parity_color",
        ],
    )
    read_files.append(day)

# Concatenate the list of dataframes into a single dataframe
m1_all = pd.concat(read_files)
# m1_all

  0%|          | 0/153 [00:00<?, ?it/s]

##### Import of flightlist data

In [3]:
flightlist = pd.read_csv(
    "/store/Projects_CRM/RAD_paper/RAD_new_data/flight_list_20240501_20241001.csv"
)
# flightlist

***
### Preprocessing of data
***

##### Preprocessing of M1 data

In [4]:
# Convert the lat and lon columns decimal minutes to decimal degrees
m1_all["segment_start_lat"] = m1_all["segment_start_lat"] / 60
m1_all["segment_start_lon"] = m1_all["segment_start_lon"] / 60
m1_all["segment_stop_lat"] = m1_all["segment_stop_lat"] / 60
m1_all["segment_stop_lon"] = m1_all["segment_stop_lon"] / 60

# Convert date and time columns to datetime
m1_all["segment_start_t"] = pd.to_datetime(
    m1_all["segment_start_t"].astype(str), format="%H%M%S"
).dt.time
m1_all["segment_stop_t"] = pd.to_datetime(
    m1_all["segment_stop_t"].astype(str), format="%H%M%S"
).dt.time
m1_all["segment_start_d"] = pd.to_datetime(
    m1_all["segment_start_d"].astype(str), format="%y%m%d"
).dt.date
m1_all["segment_stop_d"] = pd.to_datetime(
    m1_all["segment_stop_d"].astype(str), format="%y%m%d"
).dt.date

##### Preprocessing of flightlist data

In [5]:
# Rename columns for clarity
flightlist = flightlist.rename(
    columns={
        "ID": "identifier",
        "WK_TBL_CAT": "wtc",
        "REGISTRATION": "registration",
        "AIRCRAFT_ADDRESS": "icao24",
        "AIRCRAFT_OPERATOR": "operator",
        "ICAO_FLT_TYPE": "icao_flight_type",
    }
)

***
### Data merging
***

##### Merge

In [6]:
# Merge using identifier column as reference
m1_all = m1_all.merge(
    flightlist[
        [
            "identifier",
            "wtc",
            "registration",
            "icao24",
            "operator",
            "icao_flight_type",
        ]
    ],
    left_on="identifier",
    right_on="identifier",
    how="left",
)

# Reorder columns for clarity
m1_all = m1_all[
    [
        "identifier",
        "callsign",
        "operator",
        "registration",
        "ac_type",
        "origin",
        "destination",
        "icao_flight_type",
        "wtc",
        "sequence",
        "segment_id",
        "segment_length_nm",
        "segment_status",
        "segment_parity_color",
        "segment_start_d",
        "segment_start_t",
        "segment_start_fl",
        "segment_start_lat",
        "segment_start_lon",
        "segment_stop_d",
        "segment_stop_t",
        "segment_stop_fl",
        "segment_stop_lat",
        "segment_stop_lon",
    ]
]
m1_all

,identifier,callsign,operator,registration,ac_type,origin,destination,icao_flight_type,wtc,sequence,...,segment_start_d,segment_start_t,segment_start_fl,segment_start_lat,segment_start_lon,segment_stop_d,segment_stop_t,segment_stop_fl,segment_stop_lat,segment_stop_lon
0,271656154,SWR81C,SWR,HBJCE,BCS3,EGLL,LSZH,S,M,1,...,2024-05-01,05:14:00,1,51.477500,-0.461389,2024-05-01,05:14:10,5,51.475000,-0.447500
1,271656154,SWR81C,SWR,HBJCE,BCS3,EGLL,LSZH,S,M,2,...,2024-05-01,05:14:10,5,51.475000,-0.447500,2024-05-01,05:14:34,15,51.472778,-0.433611
2,271656154,SWR81C,SWR,HBJCE,BCS3,EGLL,LSZH,S,M,3,...,2024-05-01,05:14:34,15,51.472778,-0.433611,2024-05-01,05:14:46,20,51.470278,-0.419722
3,271656154,SWR81C,SWR,HBJCE,BCS3,EGLL,LSZH,S,M,4,...,2024-05-01,05:14:46,20,51.470278,-0.419722,2024-05-01,05:16:05,60,51.450833,-0.309167
4,271656154,SWR81C,SWR,HBJCE,BCS3,EGLL,LSZH,S,M,5,...,2024-05-01,05:16:05,60,51.450833,-0.309167,2024-05-01,05:20:22,60,51.378333,0.106389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308937,277038637,EZY93NG,EZY,GEZWB,A320,LSZH,EGKK,S,M,49,...,2024-09-30,22:11:04,35,51.030833,0.071389,2024-09-30,22:12:09,25,51.073333,-0.023889
308938,277038637,EZY93NG,EZY,GEZWB,A320,LSZH,EGKK,S,M,50,...,2024-09-30,22:12:09,25,51.073333,-0.023889,2024-09-30,22:12:46,20,51.089444,-0.059444
308939,277038637,EZY93NG,EZY,GEZWB,A320,LSZH,EGKK,S,M,51,...,2024-09-30,22:12:46,20,51.089444,-0.059444,2024-09-30,22:13:26,15,51.110833,-0.106944
308940,277038637,EZY93NG,EZY,GEZWB,A320,LSZH,EGKK,S,M,52,...,2024-09-30,22:13:26,15,51.110833,-0.106944,2024-09-30,22:14:05,10,51.121389,-0.130833


##### Save

In [9]:
m1_all.to_parquet(
    "/store/Projects_CRM/RAD_paper/RAD_new_data/flightplans_complete.parquet"
)